## 9. Testing and Debugging

### 79 Encapsulate Dependencies to Facilitate Mocking and Testing

In [1]:
import logging

In [2]:
class ZooDatabase:

    def get_animals(self, species):
        pass

    def get_food_period(self, species):
        pass

    def feed_animal(self, name, when):
        pass

In [3]:
from datetime import datetime

In [4]:
def do_rounds(database, species, *, utcnow=datetime.utcnow):
    now = utcnow()
    feeding_timedelta = database.get_food_period(species)
    animals = database.get_animals(species)
    fed = 0

    for name, last_mealtime in animals:
        if (now - last_mealtime) >= feeding_timedelta:
            database.feed_animal(name, now)
            fed += 1

    return fed

In [5]:
from unittest.mock import Mock

In [6]:
database = Mock(spec=ZooDatabase)
print(database.feed_animal)
database.feed_animal()
database.feed_animal.assert_any_call()

<Mock name='mock.feed_animal' id='140002176893328'>


In [7]:
from datetime import timedelta

In [8]:
from unittest.mock import call

In [9]:
now_func = Mock(spec=datetime.utcnow)
now_func.return_value = datetime(2019, 6, 5, 15, 45)

database = Mock(spec=ZooDatabase)
database.get_food_period.return_value = timedelta(hours=3)
database.get_animals.return_value = [
    ('Spot', datetime(2019, 6, 5, 11, 15)),
    ('Fluffy', datetime(2019, 6, 5, 12, 30)),
    ('Jojo', datetime(2019, 6, 5, 12, 55))
]

In [10]:
result = do_rounds(database, 'Meerkat', utcnow=now_func)
assert result == 2

In [11]:
database.get_food_period.assert_called_once_with('Meerkat')
database.get_animals.assert_called_once_with('Meerkat')
database.feed_animal.assert_has_calls(
    [
        call('Spot', now_func.return_value),
        call('Fluffy', now_func.return_value),
    ],
    any_order=True)

In [12]:
try:
    database.bad_method_name()
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-12-721e698bfb6c>", line 2, in <module>
    database.bad_method_name()
  File "/usr/local/lib/python3.8/unittest/mock.py", line 637, in __getattr__
    raise AttributeError("Mock object has no attribute %r" % name)
AttributeError: Mock object has no attribute 'bad_method_name'


In [13]:
DATABASE = None

def get_database():
    global DATABASE
    if DATABASE is None:
        DATABASE = ZooDatabase()
    return DATABASE

def main(argv):
    database = get_database()
    species = argv[1]
    count = do_rounds(database, species)
    print(f'Fed {count} {species}(s)')
    return 0

In [14]:
from unittest.mock import patch

In [15]:
import contextlib
import io

In [16]:
with patch('__main__.DATABASE', spec=ZooDatabase):
    now = datetime.utcnow()

    DATABASE.get_food_period.return_value = timedelta(hours=3)
    DATABASE.get_animals.return_value = [
        ('Spot', now - timedelta(minutes=4.5)),
        ('Fluffy', now - timedelta(hours=3.25)),
        ('Jojo', now - timedelta(hours=3)),
    ]

    fake_stdout = io.StringIO()
    with contextlib.redirect_stdout(fake_stdout):
        main(['program name', 'Meerkat'])

    found = fake_stdout.getvalue()
    expected = 'Fed 2 Meerkat(s)\n'

    assert found == expected

> - 단위 테스트를 작성할 때 목을 만들기 위해 반복적인 준비 코드를 많이 사용해야 한다면, 테스트 대상이 의존하는 다른 기능들을 더 쉽게 모킹할 수 있는 클래스로 캡슐화하는 것이 좋다.
> - `unittest.mock` 내장 모듈의 `Mock` 클래스는 클래스를 시뮬레이션할 수 있는 새로운 목을 반환한다. 이 목은 목 메서드처럼 작동할 수 있고 클래스 내 각각의 애트리뷰트에 접근할 수도 있다.
> - 끝에서 끝까지 테스트를 위해서는 테스트에 사용할 목 의존 관계를 주입하는 데 명시적인 연결점으로 쓰일 수 있는 헬퍼 함수를 더 많이 포함하도록 코드를 리팩토링하는 것이 좋다.